# Pose visualizations VTTI airport
- colmap results for VTTI airport
- pose estimations, sparse map results

In [1]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go 
from scipy.spatial.transform import Rotation as R
from colmapParsingUtils import *
import copy

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt

import cv2

%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


In [2]:
# Import colmap files

images_colmap = read_images_text('VTTI_data/MED/images.txt')
cameras = read_cameras_text('VTTI_data/MED/cameras.txt')
pts3d = read_points3D_text('VTTI_data/MED/points3D.txt')

In [3]:
def get_pose_id_mod(id):
    """
    Get the pose transformation for a specific image id
    Input: Image ID
    Output: transform from camera to world coordinates
    """
    # Get quaternion and translation vector
    qvec = images_colmap[id].qvec
    tvec = images_colmap[id].tvec[:,None]
    # print(tvec)
    t = tvec.reshape([3,1])
    # Create rotation matrix
    Rotmat = qvec2rotmat(qvec) # Positive or negative does not matter
    # print("\n Rotation matrix \n", Rotmat)
    # Create 4x4 transformation matrix with rotation and translation
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    w2c = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(w2c)
    # self.im_pts_2d[imnum]['w2c'] = w2c
	# self.im_pts_2d[imnum]['c2w'] = c2w
    return w2c, c2w

In [4]:
# Grab all poses 
# print(gnav.images_c)
poses = np.zeros([len(images_colmap),4,4])
skip_indices = {0, 11, 18, 66, 84, 85}

for n in range(len(images_colmap)):
    # print(n)
    if n in skip_indices:
        continue
    __, c2w = get_pose_id_mod(n)
    # print(c2w)
    poses[n] = c2w

print(poses)

# NOTE:
# Empty pose 0
# Empty pose 11
# Empty pose 18
# Empty pose 66
# Empty pose 84
# Empty pose 85

[[[ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.        ]]

 [[ 0.79455326  0.07143469  0.60297778  0.27268257]
  [-0.17586021  0.97756604  0.11592165  0.39475428]
  [-0.58116977 -0.19814573  0.7892908   1.49041468]
  [ 0.          0.          0.          1.        ]]

 [[ 0.79369488  0.07205231  0.60403386  0.27281595]
  [-0.17640134  0.977556    0.11518175  0.39478827]
  [-0.58217781 -0.19797154  0.78859132  1.49061506]
  [ 0.          0.          0.          1.        ]]

 ...

 [[ 0.86280754  0.0636791   0.50150585 -0.22504389]
  [-0.16155486  0.97474613  0.15417528 -0.50212471]
  [-0.47902315 -0.2140443   0.85130597 -2.22118709]
  [ 0.          0.          0.          1.        ]]

 [[ 0.86242613  0.06321548  0.50222005 -0.14685916]
  [-0.16194798  0.97447971  0.15544176 -0.47376719]
  [-0.47957692 -0.21539056  0.85065438 -2.07997979

In [5]:
# Grab raw point cloud and rgb data from scene 

raw_pts = [pts3d[key].xyz for key in pts3d.keys()]
raw_rgb = [pts3d[key].rgb for key in pts3d.keys()]
# print(raw_pts)
scene_pts = np.vstack(raw_pts)
scene_rgb = np.vstack(raw_rgb)
# Normalize
scene_rgb = scene_rgb / 255
print(scene_pts)
print(scene_rgb)

[[ 1.12940825  2.71022215  9.07917009]
 [ 1.44768681  3.15658229  8.23648973]
 [ 4.23380216  3.07969941  7.54482844]
 ...
 [-0.37457069  0.09393494 10.70742001]
 [-1.41622541  0.51846377 10.61861868]
 [-1.28204617  0.50700051 10.61118551]]
[[0.02352941 0.02745098 0.00784314]
 [0.03137255 0.03137255 0.        ]
 [0.         0.         0.        ]
 ...
 [0.43529412 0.38039216 0.22352941]
 [0.14901961 0.12941176 0.10588235]
 [0.09019608 0.0627451  0.03137255]]


In [6]:
print(len(poses))

135


In [7]:
# PLOT INITIAL POINT CLOUD AND POSES 
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud')#, width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5)

# loop through poses 
# for pose in poses:
for i in range(len(poses)):
    if (poses[i] == 0).all():
        print("Empty pose", i)
        continue
# for i in range(1):
    # print(i)
    homog_t = poses[i]
    # homog_t = pose
    # print(homog_t)
    s = i*.01
    # print(s)
    # if i in np.arange(106,135):
    if i in np.array([107, 112, 117, 122, 127, 132]):
        axes1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1).transform(homog_t)
        vis.add_geometry(axes1)
    axes1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5).transform(homog_t)
    # axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    
scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(scene_rgb)

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2


# Run the visualizer
vis.run()
vis.destroy_window()


Empty pose 0
Empty pose 11
Empty pose 18
Empty pose 66
Empty pose 84
Empty pose 85


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
pci id for fd 74: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


### FRAMES IN USE: 107, 112, 117, 122, 127, 132

- frame_31570.jpg
- frame_31620.jpg
- frame_31670.jpg
- frame_31720.jpg
- frame_31770.jpg
- frame_31820.jpg

see <vtti_gnav_demo.ipynb>

In [148]:
print(images_colmap[135].name)

frame_31840.jpg


In [149]:
for elt in np.array([107, 112, 117, 122, 127, 132]):
    print(images_colmap[elt+1].name)

frame_31570.jpg
frame_31620.jpg
frame_31670.jpg
frame_31720.jpg
frame_31770.jpg
frame_31820.jpg


In [136]:
# Determining ground plane points 

gnd_pts_IDX = np.array([1667, 1675, 1680, 1681, 1682, 1683, 1875, 1876, 1877, 12776, 12778, 12780, 12782, 12784, 12787, 12789])

arr2 = np.arange(12775,12790)
arr1 = np.array([1])
arr3 = np.array([12787, 12789])
gnd_pts_TEST = np.concatenate((arr1, arr2))

In [137]:
print(len(scene_pts))
print(gnd_pts_TEST)
print(scene_pts[gnd_pts_TEST])

30494
[    1 12775 12776 12777 12778 12779 12780 12781 12782 12783 12784 12785
 12786 12787 12788 12789]
[[ 1.44768681  3.15658229  8.23648973]
 [ 0.26401975  0.41131484  1.54531511]
 [-0.03653655 -0.06782255 -1.87108607]
 [ 0.29401562  0.39970383  1.50908626]
 [-0.18519407 -0.01817601 -1.7155522 ]
 [ 0.29388074  0.39972872  1.50920177]
 [ 0.28200253 -0.09702349 -1.92057832]
 [ 0.29135988  0.40035     1.51127481]
 [ 0.35324464 -0.1015492  -1.93511367]
 [ 0.29413734  0.39966912  1.50906061]
 [ 0.72757166 -0.12574995 -2.01179678]
 [ 0.29517006  0.39960742  1.50898387]
 [ 0.28225712  0.39720548  1.49524819]
 [ 0.04208403 -0.05476814 -1.77231044]
 [ 0.28638918  0.40202835  1.51661508]
 [ 0.44989832 -0.10897952 -1.94288102]]


In [139]:
# PLOT INITIAL POINT CLOUD AND POSES 
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud')#, width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.5)

scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(scene_rgb)

gnd_pts_t = o3d.geometry.PointCloud()
gnd_pts_t.points = o3d.utility.Vector3dVector(scene_pts[gnd_pts_TEST])
gnd_pts_t.paint_uniform_color([1,0,0])
# gnd_pts_t.colors = o3d.utility.Vector3dVector(scene_rgb[gnd_pts_TEST])

gnd_pts = o3d.geometry.PointCloud()
gnd_pts.points = o3d.utility.Vector3dVector(scene_pts[gnd_pts_IDX])
gnd_pts.paint_uniform_color([0,1,0])


vis.add_geometry(axes)
# vis.add_geometry(scene_cloud)
# vis.add_geometry(gnd_pts_t)
vis.add_geometry(gnd_pts)

# # Size options (jupyter gives issues when running this multiple times, but it looks better)
# render_option = vis.get_render_option()
# render_option.point_size = 2


# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 74
pci id for fd 74: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


[ 96  97  98  99 100 101 102 103 104 105]


In [10]:
# # GIF HELP

# # Capture frames for GIF
# frames = []
# num_frames = 30  # Adjust the number of frames
# angle_step = 180/num_frames


# for i in range(num_frames):
# 	# Rotate the view
#     view_control.rotate(angle_step, 0)  # (horizontal, vertical)

#     # vis.update_geometry(axis_orig) # Only if I move it myself?
#     vis.poll_events()
#     vis.update_renderer()

#     # Capture frame directly into memory
#     image = vis.capture_screen_float_buffer(False)
#     image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
#     frames.append(image_8bit)


# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(angle_step, angle_step/5)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
#     view_control.rotate(-angle_step, -angle_step/5)  # (horizontal, vertical)

#     # vis.update_geometry(axis_orig) # Only if I move it myself?
#     vis.poll_events()
#     vis.update_renderer()

#     # Capture frame directly into memory
#     image = vis.capture_screen_float_buffer(False)
#     image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
#     frames.append(image_8bit)


# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# for i in range(num_frames):
# 	# Rotate the view
# 	view_control.rotate(-angle_step, 0)  # (horizontal, vertical)

# 	# vis.update_geometry(axis_orig) # Only if I move it myself?
# 	vis.poll_events()
# 	vis.update_renderer()

# 	# Capture frame directly into memory
# 	image = vis.capture_screen_float_buffer(False)
# 	image_8bit = (np.asarray(image) * 255).astype(np.uint8)  # Convert to 8-bit
# 	frames.append(image_8bit)

# # Create GIF
# # Ensure frames are in the correct format
# frames = [frame.astype("uint8") for frame in frames]

# # Use imageio to save as GIF
# imageio.mimsave("test.gif", frames, fps=30, loop=0)  # Adjust fps if necessary